### Notes
- I need to convert colors to grayscale intensities, then replace them with whatever predefined color is closest within the given palette (color quantization).

- I should create a grayscale version of good icon set, then quantize it, and have that serve as the base. I can keep a folder of different icon bases, which would only serve to provide geometry.

- I should rid the icon svg files of metadata.

- Don't use xml.etree.ElementTree for finding colors in svg. Instead save the contents as a string and search it for "fill:#XXXXXX" using regex and store in a set to avoid repeat colors.

### Functions

In [15]:
import xml.etree.ElementTree as ET
import yaml, os

def replace_color(path, old, new):
    """Replaces hex strings representing colors in files."""
    with open(path, 'r') as file:
        content = file.read()

    modified = content.replace("fill:" + old, new)

    with open(path, 'w') as file:
        file.write(modified)

def load_palette(path):
    """Load a palette.yaml file."""
    with open(path, 'r') as file:
        palette = yaml.safe_load(file)

    return palette

def extract_fill(style):
    """Extract fill from style attribute within svg elements."""
    properties = style.split(';')
    for prop in properties:
        key, value = prop.split(':')
        if key.strip() == 'fill':
            return value.strip()
        
def get_fill_colors(path):
    """Return a list of all fill colors within an svg."""
    tree = ET.parse(path)
    root = tree.getroot()
    fill_colors = set()

    # Recursive function to traverse SVG elements.
    def traverse_elements(element):
        if 'fill' in element.attrib:
            fill_color = element.attrib['fill']
            if fill_color not in fill_colors:
                fill_colors.add(fill_color)

        if 'style' in element.attrib:
            style = element.attrib['style']
            fill_property = extract_fill(style)
            if fill_property and fill_property not in fill_colors:
                fill_colors.add(fill_property)

        for child in element:
            traverse_elements(child)

    traverse_elements(root)
    return list(fill_colors)

def hex_color_to_grayscale(hex):
    """Convert a hexadecimal color to a grayscale equivalent."""
    hex = hex.lstrip('#')
    r,g,b = int(hex[0:2],16), int(hex[2:4],16), int(hex[4:6],16)
    gs = int(0.21*r + 0.72*g + 0.07*b)
    hex_gs = '#' + format(gs, '02x')*3
    return hex_gs

def get_icon_paths(folder_path):
    paths = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.svg'):
            paths.append(os.path.join(folder_path, file_name))
    return paths


### Function Tests

In [9]:
# Test replace_color().
palette_path = "palettes/nord.yaml"
icon_path = "test_icons/firefox.svg"
old = "hello"; new = "world"

In [16]:
# Test load_palette().
palette = load_palette(palette_path)
print(palette["name"])
print(palette["colors"])

Nord dark mono
['#4f4f4f', '#3d4555', '#4c566a', '#5f697d', '#737d91', '#cddff0', '#afafb1', '#ffffff']


In [7]:
# Test get_fill_colors().
file_path = 'test_icons/firefox.svg'
fill_colors = get_fill_colors(file_path)
print(fill_colors)

['#5f697d', '#ffffff', '#4c566a']


In [19]:
# Test hex_to_grayscale() and grayscale_to_hex().
hex_color_to_grayscale("#FF0000") # Red.

'#353535'

In [20]:
# Test get_icon_paths().
folder_path = 'test_icons'
svg_paths = get_icon_paths(folder_path)
print(svg_paths)

['test_icons/firefox.svg', 'test_icons/obsidian.svg', 'test_icons/visual-studio-code.svg']


### Full Tests

In [27]:
def modify_copy(path, prefix, modification):
    """Open a file at the given path, then apply modifications defined
    in a function, save a copy, prefixing its name with a string"""

    with open(path, 'r') as file:
        content = file.read()
        content = modification(content)

    with open(prefix + os.path.basename(path), 'w') as new_file:
        new_file.write(content)

In [25]:
def grayscale(path):
    colors = get_fill_colors() # Rewrite (see notes).
    for color in colors:
        gs = hex_color_to_grayscale(color)
        print(gs) 

modify_copy("test_icons/firefox.svg", "gray_", grayscale)